In [1]:
import pandas as pd
import seaborn as sb

# 1 Select Data

In [2]:
cot_all = pd.read_excel("..\\..\\Datos\\working\\cotizaciones_all.xlsx")
neg_all = pd.read_excel("..\\..\\Datos\\working\\negocios_all.xlsx")

In [3]:
# cot_all.columns

In [4]:
cot_all.drop(['Remoto','Etapa','Tipo de Medio','Razon Social','Cargo',
              'Situacion Laboral','Antiguedad Laboral'],
            axis=1,inplace=True)

In [5]:
cot_mod = cot_all.copy()

In [6]:
# ruts de los clientes en negocio
neg_ids = neg_all['ID Cotizacion'].unique().tolist()

# cot_neg = cot_all.copy()
cot_mod['Negocio'] = cot_all['ID'].isin(neg_ids)
cot_mod[cot_mod['Negocio'] == True].shape

(2336, 35)

# 2 Clean Data

In [7]:
# variables a impiar y chequear

# Limpiar variable fecha de nacimiento de alguna manera
# Limpiar variable Sexo usando rutificador
# Limpar variables Direccion usando en lo posible el rutificador y otras fuentes
# Llenar mv en decuentos con 0

## 2.1 Rut
Arreglar Ruts, luego validarlos para aplicar data crawling en rutificador.
Si el rut esta mal escrito por ende no es valido, se prosigue a buscar el rut por el nombre y arreglar el rut.
Todo esto con el find de obtener el genero, direccion, latiyud y longitud del cotizante

### 2.1.1 Funciones
Funciones para tratar los ruts

In [8]:
def format_rut(x):
    rut = x.split('-')
    l_s = list(rut[0])
    r_l = l_s[::-1]
    new = []
    i = 0
    new.append(rut[1])
    new.append('-')
    for c in r_l:
        if i==3:
            new.append('.')
            i = 0
        new.append(c)
        i +=1
    rut = ''.join(new[::-1])
    return rut

In [9]:
import chile_rut
import re

def val_rut(x):
    x = str(x)
    if len(x) == 0:
        return False
    x = re.sub('[.|!@#$]', '', x)
    
    if len(x.split('-')) >= 2:
        return chile_rut.validate_rut(x)
    else:
        x = x[:-1] + '-' + x[-1]
        return chile_rut.validate_rut(x)

def clean_rut(x):
    x = str(x)
    if len(x) == 0:
        return False
    x = re.sub('[.|!@#$]', '', x)
    
    if len(x.split('-')) >= 2:
        return x
    else:
        x = x[:-1] + '-' + x[-1]
        return x
    
cot_mod['Clean Rut'] = [clean_rut(x) for x in cot_mod['RUT'].values]
cot_mod['Valid Rut'] = [val_rut(x) for x in cot_mod['Clean Rut'].values]


### 2.1.1 Insights
Revision de los ruts, mal formaciones, ruts malos que tienen negocios, etc

In [10]:
#Eliminar indice 1704 y arreglar rut max boris
print('Ruts que hicieron negocios pero no son validos')
cot_mod[(cot_mod['Valid Rut'] == False) & (cot_mod['Negocio'] == True)][['Nombre','RUT','Negocio','Valid Rut','Clean Rut']]

Ruts que hicieron negocios pero no son validos


,Nombre,RUT,Negocio,Valid Rut,Clean Rut
7390,max boris,14030274-6,True,False,14030274-6
7504,max boris,14030274-6,True,False,14030274-6
1704,Error,15516786-8,True,False,15516786-8


In [11]:
# ver que ruts invalidos pueden ser rescatados o arreglados buscando por sus nombres
print("Ruts catalogados como invalidos, buscar por nombres")
ruts_iv = cot_mod[(cot_mod['Valid Rut'] == False)] 
print(ruts_iv.shape)

ruts_invalidos = []
for group, frame in  ruts_iv.groupby('Clean Rut'):
    ruts_invalidos.append(str(group))

s_ruts = pd.Series(ruts_invalidos)
s_ruts.to_csv("..\\..\\Datos\\ruts_invalidos_scrap.csv")

# print(ruts_iv[['Nombre','Apellido 1','RUT','Clean Rut','Valid Rut']].head())

Ruts catalogados como invalidos, buscar por nombres
(69, 37)


In [12]:
print("Ruts arreglados, estaban mal escritos y luego fueron validos.")
df = cot_mod[(cot_mod['RUT'] != cot_mod['Clean Rut']) & (cot_mod['Valid Rut'] ==True)]
print(df.shape)
print(df[['Nombre','Apellido 1','Apellido 2','RUT','Clean Rut','Valid Rut']].head(10))

Ruts arreglados, estaban mal escritos y luego fueron validos.
(1, 37)
      Nombre Apellido 1 Apellido 2        RUT   Clean Rut  Valid Rut
8  Valentina  Fernandez          .  19483862k  19483862-k       True


In [13]:
print("Ruts arreglados, estaban mal escritos y luego fueron validos.")
df = cot_mod[(cot_mod['RUT'] != cot_mod['Clean Rut']) & (cot_mod['Valid Rut'] ==False)]
print(df.shape)
print(df[['Nombre','Apellido 1','Apellido 2','RUT','Clean Rut','Valid Rut']].head(10))

Ruts arreglados, estaban mal escritos y luego fueron validos.
(6, 37)
              Nombre Apellido 1 Apellido 2           RUT   Clean Rut  \
821         Giovanni     Flores          .  16.546.158-2  16546158-2   
841         Giovanni     Flores          .  16.546.158-2  16546158-2   
3716           Mario     Concha     Concha     156778804  15677880-4   
8943    Jose Edgardo    Caamaño   Chamorro       6180535    618053-5   
9099   Dinora  Nilda      Bravo      Cerda       9559524    955952-4   
9574  Anegelo Felipe      Godoy    Morales    |7321289-5   7321289-5   

      Valid Rut  
821       False  
841       False  
3716      False  
8943      False  
9099      False  
9574      False  


In [14]:
print("Ruts que estaban bien escritos.")
df = cot_mod[(cot_mod['RUT'] == cot_mod['Clean Rut'])]
print(df.shape)

Ruts que estaban bien escritos.
(16585, 37)


In [15]:
print("Ruts que estaban bien escritos y son validos.")
df = cot_mod[(cot_mod['RUT'] == cot_mod['Clean Rut']) &(cot_mod['Valid Rut'] ==True)]
print(df.shape)

ruts_validos = []
for group, frame in  df.groupby('Clean Rut'):
    ruts_validos.append(str(group))

# print(ruts_invalidos)
ruts_validos= [format_rut(x) for x in ruts_validos]
cot_mod['Format Rut'] = [format_rut(x) for x in cot_mod['Clean Rut'].values]
s_ruts = pd.Series(ruts_validos)
s_ruts.to_csv("..\\..\\ruts_validos_scrap.csv")

Ruts que estaban bien escritos y son validos.
(16522, 37)


In [16]:
print("Ruts que estaban bien escritos pero no son validos, buscar ruts por nombres.")
df = cot_mod[(cot_mod['RUT'] == cot_mod['Clean Rut']) &(cot_mod['Valid Rut'] ==False)]
print(df.shape)

Ruts que estaban bien escritos pero no son validos, buscar ruts por nombres.
(63, 38)


### 2.1.1 Ruts Scrapped

In [17]:
#Ruts y sus datos obtenidos de forma valida
ruts_scrapped = pd.read_csv('..\\..\\DataCrawling\\rut\\ruts_validos.csv')
ruts_scrapped['RUT'] = [x.replace(' ','') for x in ruts_scrapped['RUT'].values]
ruts_scrapped['RUT'] = [x.replace('K','k') for x in ruts_scrapped['RUT'].values]
print(ruts_scrapped.shape)
print(ruts_scrapped.head(2))

(6012, 5)
                                  Name           RUT Gender  \
0   SCHNEIDER BARRERA RICARDO ERNESTO   10.023.348-7   VAR    
1     MORALES ESPINOZA CLAUDIO ANDRES   10.020.262-k   VAR    

                             Adress               Comuna  
0                J  CISTERNAS 1612            LA SERENA   
1   ALEJANDRO SEXTO 2787LZO ARENAS    CONCEPCION CENTRO   


### 2.1.1 Ruts Scrapped merge a DF principal

In [18]:
# merge
cot_mod_cp = cot_mod.copy()
cot_mod_cp['Format Rut'] = [x.replace('K','k') for x in cot_mod_cp['Format Rut'].values]
cot_merge = pd.merge(cot_mod_cp, ruts_scrapped,how='left', left_on='Format Rut', right_on='RUT')
# print(cot_merge.columns)
columns = ['Format Rut','Valid Rut','Negocio','Nombre','Apellido 1','Name','Gender','Sexo',
           'Adress','Direccion', 'Region','Provincia', 'Comuna_x', 'Comuna_y']
cot_merge[columns][cot_merge['Sexo'].isnull()].shape

(1869, 14)

In [19]:

ruts_scrapped['RUT'] = [x.replace(' ','') for x in ruts_scrapped['RUT'].values]

# df = pd.merge(cot_mod_cp,ruts_scrapped,how='outer',left_on='Format Rut',right_on='RUT')
df = pd.merge(cot_mod_cp,ruts_scrapped,how='outer',left_on='Format Rut',right_on='RUT')

ruts_scrapped.to_csv('..\\..\\Datos\\working\\ruts_scrapped_mod.csv')
df.to_excel('..\\..\\Datos\\tmp.xlsx')

In [20]:
# len(df[df['Negocio'] == True]['RUT'].unique())

## 2.2 Profesion

In [21]:
cot_mod.columns

Index(['ID', 'Fecha Cotizacion', 'Medio', 'Proyecto', 'Productos',
       'Presencial', 'Jefe Comercial', 'Ejecutivo Comercial',
       'Total Productos', 'Descuentos', 'Valor Final Venta', 'RUT', 'Nombre',
       'Apellido 1', 'Apellido 2', 'Nombre Completo', 'Telefono', 'Celular',
       'Direccion', 'Region', 'Provincia', 'Comuna', 'Correo Electronico',
       'Sexo', 'Estado Civil', 'Rango Edad', 'Fecha Nacimiento',
       'Tipo Cliente', 'Giro', 'Nacionalidad', 'N° Grupo Familiar',
       'Actividad', 'Empleador', 'Profesión', 'Negocio', 'Clean Rut',
       'Valid Rut', 'Format Rut'],
      dtype='object')

In [22]:
cot_mod['profesion_agg'] = cot_mod['Profesión']
print('Null Profesiones ',cot_mod.profesion_agg.isnull().sum())

cot_mod['profesion_agg'].fillna('sin informacion',inplace=True)
print('Null Profesiones ',cot_mod.profesion_agg.isnull().sum())

# cot_mod['profesion_area'] = cot_mod['Profesión']

cot_prof = cot_mod[~cot_mod['Profesión'].isnull()]
cot_prof =cot_prof[['Profesión','profesion_agg','RUT']]
print(len(cot_prof.RUT.unique()))
print(len(cot_prof.profesion_agg.unique()))

9226
0
2251
129


In [23]:
# accented_string = u'Málaga'
# # accented_string is of type 'unicode'
import unidecode
# unaccented_string = unidecode.unidecode(accented_string)
# unaccented_string.lower()

# cot_mod['profesion_agg'].fillna('sin informacion')
# cot_mod['profesion_agg'] = [unidecode.unidecode(x).lower() for x in cot_mod['profesion_agg']]
# # cot_mod.profesion_agg = cot_prof.profesion_agg

In [24]:
cot_prof.head()

,Profesión,profesion_agg,RUT
3,Empresario,Empresario,7188659-k
4,Constructor Civil,Constructor Civil,13377606-0
9,Traductor e Interprete,Traductor e Interprete,12920930-5
10,Traductor e Interprete,Traductor e Interprete,12920930-5
11,Matrona,Matrona,10076241-2


In [25]:
# print(len(cot_prof.RUT.unique()))
# print(len(cot_prof.agg_profesion.unique()))

In [26]:
# from gensim.models import Word2Vec
# import numpy as np
# from scipy.spatial.distance import cdist

# sentences = [cot_prof.agg_profesion.tolist(), cot_prof.agg_profesion.tolist()]
# model = Word2Vec(sentences=sentences, size=10, window=1, iter=2000, min_count=1)

# _ = {word: idx for (idx, word) in enumerate(model.wv.index2word)}  # for comfort
# # assert len(_) == 6

# similarity = 1 - cdist(model.wv.syn0, model.wv.syn0, metric='cosine')
# assert similarity.shape == (6, 6)

# similarity[_["cat"], _["dog"]]  # similarity between 'cat' and 'dog'

In [27]:
# sim_pd = pd.DataFrame(data=similarity, index=cot_prof.agg_profesion.unique().tolist(),
#                       columns=cot_prof.agg_profesion.unique().tolist())

In [28]:
# sim_pd

In [29]:
cot_mod.to_csv("..\\..\\Datos\\working\\cotizaciones_to_personas.csv")
cot_mod.to_excel("..\\..\\Datos\\working\\cotizaciones_to_personas.xlsx")